In [1]:
import numpy as np
import pandas as np
import os
import matplotlib.pyplot as plt

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import f1_score
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomTranslation, GaussianNoise, Input, Rescaling
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
from google.colab import files
uploaded= files.upload()

Saving pneumoniamnist.npz to pneumoniamnist.npz


In [5]:
import numpy as np
df = np.load('pneumoniamnist.npz')

In [6]:
train_images = df['train_images']
train_labels = df['train_labels']
val_images = df['val_images']
val_labels = df['val_labels']
test_images = df['test_images']
test_labels = df['test_labels']

In [7]:
print("All datasets extracted into individual variables.")
print(f"Shape of train_images: {train_images.shape}")
print(f"Shape of train_labels: {train_labels.shape}")
print(f"Shape of val_images: {val_images.shape}")
print(f"Shape of val_labels: {val_labels.shape}")
print(f"Shape of test_images: {test_images.shape}")
print(f"Shape of test_labels: {test_labels.shape}")

All datasets extracted into individual variables.
Shape of train_images: (3882, 28, 28)
Shape of train_labels: (3882, 1)
Shape of val_images: (524, 28, 28)
Shape of val_labels: (524, 1)
Shape of test_images: (624, 28, 28)
Shape of test_labels: (624, 1)


In [8]:
import tensorflow as tf

# Define the target size for Inception-V3
IMG_HEIGHT = 299
IMG_WIDTH = 299

# Function to resize and add color channels
def preprocess_image(image):
    # Ensure image is float32 for resizing and normalization
    # This cast is crucial as the first step for consistent float processing
    image = tf.cast(image, tf.float32)

    # Add a channel dimension: (28, 28) -> (28, 28, 1)
    image = tf.expand_dims(image, axis=-1)

    # Resize image to Inception-V3's expected input size
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))

    # Convert the single channel (grayscale) to 3 channels (RGB)
    image = tf.concat([image, image, image], axis=-1)

    # Normalize pixel values to be between -1 and 1, as InceptionV3 expects
    # (Values were 0-255, now 0-1 after /255, now -1 to 1 after *2 - 1)
    image = (image / 255.0) * 2.0 - 1.0

    return image

print("Applying preprocessing to images...")

# Apply preprocessing to all image sets
# Convert numpy arrays to TensorFlow Tensors for efficient processing with map_fn
# It's good practice to ensure the input Tensors are also of the correct type if known
# However, the cast inside preprocess_image should handle it for map_fn.

# The fix: explicitly specify the dtype for the output of tf.map_fn
train_images_processed = tf.map_fn(preprocess_image, train_images, dtype=tf.float32)
val_images_processed = tf.map_fn(preprocess_image, val_images, dtype=tf.float32)
test_images_processed = tf.map_fn(preprocess_image, test_images, dtype=tf.float32)


print("Image preprocessing complete.")
print(f"New shape of train_images: {train_images_processed.shape}")
print(f"New shape of val_images: {val_images_processed.shape}")
print(f"New shape of test_images: {test_images_processed.shape}")

Instructions for updating:
Use fn_output_signature instead


Applying preprocessing to images...
Image preprocessing complete.
New shape of train_images: (3882, 299, 299, 3)
New shape of val_images: (524, 299, 299, 3)
New shape of test_images: (624, 299, 299, 3)


In [9]:
import numpy as np
from collections import Counter

# Prepare labels (ensure they are integers, typically 0 or 1)
# They are already (N, 1) but reshaping to (N,) can sometimes be more convenient
train_labels_flat = train_labels.flatten()
val_labels_flat = val_labels.flatten()
test_labels_flat = test_labels.flatten()


print("Label preparation complete.")

print("\n--- Class Imbalance Detection (Training Set) ---")
# Count occurrences of each class in the training labels
train_class_counts = Counter(train_labels_flat)

print(f"Training Set Class Distribution: {train_class_counts}")

# Assuming 0 is 'Normal' and 1 is 'Pneumonia' (or vice-versa, check your dataset documentation if unsure)
# For PneumoniaMNIST, 0 is Normal, 1 is Pneumonia.
normal_count = train_class_counts.get(0, 0)
pneumonia_count = train_class_counts.get(1, 0)

total_train_samples = len(train_labels_flat)

print(f"Normal samples in training set: {normal_count} ({normal_count / total_train_samples:.2%})")
print(f"Pneumonia samples in training set: {pneumonia_count} ({pneumonia_count / total_train_samples:.2%})")

if normal_count != 0 and pneumonia_count != 0:
    imbalance_ratio = max(normal_count, pneumonia_count) / min(normal_count, pneumonia_count)
    print(f"Imbalance ratio (majority to minority): {imbalance_ratio:.2f}:1")
    if imbalance_ratio > 1.5: # A common heuristic for considering imbalance significant
        print("Warning: Significant class imbalance detected! This will need to be addressed.")
    else:
        print("Class imbalance is minor or non-existent in the training set.")
else:
    print("Cannot calculate imbalance ratio: one or both classes are missing from the training set.")

# Also print for validation and test, though mitigation is primarily for training
print("\n--- Class Distribution (Validation Set) ---")
val_class_counts = Counter(val_labels_flat)
print(f"Validation Set Class Distribution: {val_class_counts}")

print("\n--- Class Distribution (Test Set) ---")
test_class_counts = Counter(test_labels_flat)
print(f"Test Set Class Distribution: {test_class_counts}")

Label preparation complete.

--- Class Imbalance Detection (Training Set) ---
Training Set Class Distribution: Counter({np.uint8(1): 3494, np.uint8(0): 388})
Normal samples in training set: 388 (9.99%)
Pneumonia samples in training set: 3494 (90.01%)
Imbalance ratio (majority to minority): 9.01:1

--- Class Distribution (Validation Set) ---
Validation Set Class Distribution: Counter({np.uint8(1): 389, np.uint8(0): 135})

--- Class Distribution (Test Set) ---
Test Set Class Distribution: Counter({np.uint8(1): 390, np.uint8(0): 234})


In [10]:
from sklearn.utils import class_weight
import numpy as np # Already imported, but good for context

# Assuming train_labels_flat is already defined from Step 5
# and contains the flattened training labels (0s and 1s)

# Calculate class weights
# 'balanced' mode automatically assigns weights inversely proportional to class frequencies.
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_flat),
    y=train_labels_flat
)

# Convert class_weights to a dictionary for Keras
class_weight_dict = dict(enumerate(class_weights))

print("\n--- Class Weights Calculated ---")
print(f"Original class counts: {Counter(train_labels_flat)}")
print(f"Calculated class weights: {class_weight_dict}")
print("These weights will be used during model training to mitigate imbalance.")


--- Class Weights Calculated ---
Original class counts: Counter({np.uint8(1): 3494, np.uint8(0): 388})
Calculated class weights: {0: np.float64(5.002577319587629), 1: np.float64(0.5555237550085862)}
These weights will be used during model training to mitigate imbalance.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast

print("Setting up Data Augmentation layers...")

# Define data augmentation layers
# These layers will only be applied during training
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"), # Flip images horizontally and vertically
  RandomRotation(0.2),                   # Rotate images by up to 20%
  RandomZoom(0.2),                       # Zoom in/out by up to 20%
  # You can add more as needed, e.g., RandomContrast, RandomTranslation
], name="data_augmentation")

print("Data augmentation layers defined.")
print("These will be applied to the training images during the training process.")

Setting up Data Augmentation layers...
Data augmentation layers defined.
These will be applied to the training images during the training process.


In [12]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

print("Loading pre-trained Inception-V3 model...")

# Load the InceptionV3 model with pre-trained ImageNet weights
# include_top=False means we don't include the classifier layers at the top,
# we'll add our own.
# input_shape should match the preprocessed images (299, 299, 3)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze the convolutional base
# This prevents the weights of the pre-trained layers from being updated during training.
# We want to use their learned feature extraction capabilities as they are.
base_model.trainable = False

print("Inception-V3 base model loaded and frozen.")
print("Building custom classification head...")

# Add your custom classification head on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x) # Reduces spatial dimensions to a single vector per feature map
x = Dense(128, activation='relu')(x) # A dense layer for processing extracted features
x = Dropout(0.5)(x) # Dropout for further regularization to prevent overfitting
predictions = Dense(1, activation='sigmoid')(x) # Output layer for binary classification (0 or 1 probability)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

print("Custom classification head built.")
print("Model summary:")
model.summary()

Loading pre-trained Inception-V3 model...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Inception-V3 base model loaded and frozen.
Building custom classification head...
Custom classification head built.
Model summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,065,185 (84.17 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [13]:
import tensorflow as tf

print("Compiling the model...")

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Adam is a good general-purpose optimizer
              loss=tf.keras.losses.BinaryCrossentropy(), # BinaryCrossentropy for binary classification
              metrics=[
                  'accuracy', # Overall accuracy
                  tf.keras.metrics.Precision(), # Precision metric
                  tf.keras.metrics.Recall(),    # Recall metric
                  tf.keras.metrics.AUC()        # AUC (Area Under the Curve) for ROC curve
              ])

print("Model compiled successfully!")
print("Optimizer: Adam (learning_rate=0.0001)")
print("Loss Function: BinaryCrossentropy")
print("Metrics: Accuracy, Precision, Recall, AUC")

Compiling the model...
Model compiled successfully!
Optimizer: Adam (learning_rate=0.0001)
Loss Function: BinaryCrossentropy
Metrics: Accuracy, Precision, Recall, AUC


In [20]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

print("Starting model training with Early Stopping and Model Checkpointing...")

# Define Early Stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Define Model Checkpoint callback
# Choose a path in your Google Drive to save the weights
checkpoint_dir = '/content/drive/My Drive/pneumonia_model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True) # Create the directory if it doesn't exist

# --- FIX IS ON THIS LINE ---
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_inceptionv3_pneumonia.weights.h5') # Changed filename!
# ---------------------------

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

# Number of epochs and batch size
EPOCHS = 10 # Or your preferred number of max epochs
BATCH_SIZE = 32

# Train the model
history = model.fit(
    train_images_processed,
    train_labels,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_images_processed, val_labels),
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint_callback],
    verbose=1
)

print("\nModel training complete.")
print("Training history (loss and metrics) stored in 'history' object.")
print(f"Best model weights saved to: {checkpoint_filepath}")

Starting model training with Early Stopping and Model Checkpointing...
Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8834 - auc: 0.9593 - loss: 0.2722 - precision: 0.9862 - recall: 0.8812
Epoch 1: val_loss improved from inf to 0.21181, saving model to /content/drive/My Drive/pneumonia_model_checkpoints/best_inceptionv3_pneumonia.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 1143s 9s/step - accuracy: 0.8835 - auc: 0.9593 - loss: 0.2720 - precision: 0.9862 - recall: 0.8813 - val_accuracy: 0.9179 - val_auc: 0.9712 - val_loss: 0.2118 - val_precision: 0.9626 - val_recall: 0.9254
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9141 - auc: 0.9724 - loss: 0.2118 - precision: 0.9916 - recall: 0.9125
Epoch 2: val_loss did not improve from 0.21181
122/122 ━━━━━━━━━━━━━━━━━━━━ 1123s 9s/step - accuracy: 0.9140 - auc: 0.9724 - loss: 0.2118 - precision: 0.9916 - recall: 0.9124 - val_accuracy: 0.9198 - val_auc: 0.9722 - val_loss: 0.2291 - val_precision: 0.9833 - val_recal

In [21]:
import tensorflow as tf
import os
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score
import numpy as np

# Define the checkpoint path (ensure this matches what was saved)
checkpoint_dir = '/content/drive/My Drive/pneumonia_model_checkpoints'
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_inceptionv3_pneumonia.weights.h5')

# Load the best weights into your defined model
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Successfully loaded best model weights from checkpoint (Epoch 10's best performance).")
else:
    print("Error: No checkpoint file found at the specified path. Cannot perform evaluation.")

print("\nEvaluating model on the test set...")

# Evaluate the model on the pre-processed test images and labels
test_loss, test_accuracy, test_auc, test_precision, test_recall = model.evaluate(
    test_images_processed,
    test_labels,
    verbose=1 # Show progress
)

# Predict probabilities on the test set
test_predictions_proba = model.predict(test_images_processed)

# Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
test_predictions_binary = (test_predictions_proba > 0.5).astype(int)

# Calculate additional metrics using scikit-learn for clarity, especially F1-Score and Balanced Accuracy
# Ensure test_labels are flat if they are (N, 1) and predictions are flat
y_true_flat = test_labels.flatten()
y_pred_flat = test_predictions_binary.flatten()

# Scikit-learn F1-score
f1 = f1_score(y_true_flat, y_pred_flat)

# Scikit-learn Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_true_flat, y_pred_flat)

print("\n--- Model Evaluation Results on Test Set ---")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy (Keras): {test_accuracy:.4f}")
print(f"Test AUC (Keras): {test_auc:.4f}")
print(f"Test Precision (Keras): {test_precision:.4f}")
print(f"Test Recall (Keras): {test_recall:.4f}")

print(f"F1-Score (scikit-learn): {f1:.4f}")
print(f"Balanced Accuracy (scikit-learn): {balanced_accuracy:.4f}")

print("\nEvaluation complete. These are your model's final performance metrics.")

Successfully loaded best model weights from checkpoint (Epoch 10's best performance).

Evaluating model on the test set...
20/20 ━━━━━━━━━━━━━━━━━━━━ 159s 8s/step - accuracy: 0.8745 - auc: 0.9490 - loss: 0.3289 - precision: 0.8519 - recall: 0.9619
20/20 ━━━━━━━━━━━━━━━━━━━━ 163s 8s/step

--- Model Evaluation Results on Test Set ---
Test Loss: 0.3168
Test Accuracy (Keras): 0.8718
Test AUC (Keras): 0.8555
Test Precision (Keras): 0.9564
Test Recall (Keras): 0.9495
F1-Score (scikit-learn): 0.9031
Balanced Accuracy (scikit-learn): 0.8436

Evaluation complete. These are your model's final performance metrics.
